In [20]:
# Dependencies and Setup
import requests
import gmaps
import pandas as pd

# Import API key
from config import g_key

In [34]:
hotels_df=pd.read_csv("prefered_weather.csv")
hotels_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Marrakesh,MA,84.27,31.6315,-8.0083,NaN
1,Buchanan,LR,81.21,5.8808,-10.0467,NaN
2,Safford,US,81.10,32.8340,-109.7076,NaN
3,Kharakvasla,IN,80.31,18.4333,73.7667,NaN
4,Hilo,US,77.29,19.7297,-155.0900,NaN


In [35]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uECw9_qYFqhI2vk87kF8PnW_ApYruCzR7JD9--JPkNunIVBcF-HK1dFOPWbYULAXNT9OJT6eNbXWE9PUX_whgEGp8crj4WOyUvKncnUAMrB7KuAmlMWMNZtZKq_l13h2xs3DV6VFDD8ChS_xlxYzpUzwf9zZbVyzbHoWx8BdtjsB9WMA5vc7k-B20fn8qZYupD7tJzYkm30yorHTrkInFUkixRjur5p-In3lE1CiOZlYPwTDew7TRzPGvL5oaJbXHn4crZlsTxLBr7qausfhLV2CGocQrjBC8c18c5F9kweDh7GbEWWpvaq7Ejx3pDAc7Y2vzutjDy1bAhJSRYReHWVhq6PYsPEBTn-60MePDE3vHoKIblkg57cI0Jc2e04xsSBTTgSrFjY_Pb1e7vvSPvgu4RKtRZt5eX0pR8TLihf2Y1_JpbRwzjcD',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [36]:
# Iterate through the DataFrame.
for index, row in hotels_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotels_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
hotels_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Marrakesh,MA,84.27,31.6315,-8.0083,Les Jardins de La Koutoubia
1,Buchanan,LR,81.21,5.8808,-10.0467,Kamaneahn Hotel Inc
2,Safford,US,81.10,32.8340,-109.7076,SpringHill Suites by Marriott Thatcher
3,Kharakvasla,IN,80.31,18.4333,73.7667,Savali Resort
4,Hilo,US,77.29,19.7297,-155.0900,Hilo Hawaiian Hotel


In [52]:
# Add a heatmap of temperature for the vacation spots.
locations = hotels_df[["Lat", "Lng"]]
max_temp = hotels_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))